In [1]:
import findspark
findspark.init('C:\Spark\spark-3.0.2-bin-hadoop2.7')
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc

In [2]:
sc = SparkContext()
ssc = StreamingContext(sc, 10 )
sqlContext = SQLContext(sc)

In [3]:
socket_stream = ssc.socketTextStream("127.0.0.1", 5556)
lines = socket_stream.window( 20 )

In [4]:
from collections import namedtuple
fields = ("tag", "count" )
Tweet = namedtuple('tweets', fields )

In [5]:

( lines.flatMap( lambda text: text.split( " " ) ) #Splits to a list
  .filter( lambda word: word.lower().startswith("#") ) # Checks for hashtag calls
  .map( lambda word: ( word.lower(), 1 ) ) # Lower cases the word
  .reduceByKey( lambda a, b: a + b ) # Reduces
  .map( lambda rec: Tweet( rec[0], rec[1] ) ) # Stores in a Tweet Object
  .foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") ) # Sorts Them in a DF
  .limit(10).createOrReplaceGlobalTempView("tweets") ) ) # Registers to a table.

In [6]:
import time
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns
import pandas
# Only works for Jupyter Notebooks!
%matplotlib inline

In [7]:
ssc.start()

In [8]:
count = 0
while count < 10:
    
    time.sleep( 3 )
    top_10_tweets = sqlContext.sql( 'Select tag, count from global_temp.tweets' )
    top_10_df = top_10_tweets.toPandas()
    #top_10_df.sort(desc("count"))
    #print(top_10_df)
    display.clear_output(wait=True)
    plt.figure( figsize = ( 10, 8 ) )
    sns.barplot( x="count", y="keyword", data=top_10_df)
    plt.show()
    count = count + 1
    print (count)

AnalysisException: Table or view not found: global_temp.tweets; line 1 pos 23;
'Project ['tag, 'count]
+- 'UnresolvedRelation [global_temp, tweets]


In [10]:
ssc.stop()